<div style="text-align: center;">
    <center><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ3Zwg8uptvuP3WYPy5qHUK1qllvZx5dxghGA&s" />
</div>

# <center>**Artificial Intelligence**
# <center>Connect 4 Project
## <center>AI Instructor
## <center>Eng. Yousef Elbaroudy
## <center>Team Members
### <center> سكشن 2
### <center> محمد امير ابوزيد احمد
### <center> محمد ايمن يحيى عبد السلام
###  <center> سكشن 3
### <center> محمد صبحي محمد عواد
### <center> يوسف هشام عبد الفتاح عبد العظيم
## <center>2025/2026

________________________________________________________________________________________

## <center> AI VS AI

### Importing Libraries and Defining Constants
We import `numpy` for the board representation, `random` for random choices, and `math` for infinity values.  
We also define constants for the board dimensions, player pieces, and window length.


In [9]:
import numpy as np
import random
import math

ROW_COUNT = 6
COLUMN_COUNT = 7

AI1 = 0
AI2 = 1

EMPTY = 0
AI1_PIECE = 1
AI2_PIECE = 2

WINDOW_LENGTH = 4


### Board Creation and Printing
- `create_board()` creates an empty board filled with zeros.
- `print_board()` flips the board vertically for a more natural view when printed.


In [10]:
def create_board():
    return np.zeros((ROW_COUNT, COLUMN_COUNT))

def print_board(board):
    print(np.flip(board, 0))


### Checking Valid Moves and Dropping Pieces
- `is_valid_location(board, col)` checks if the top row of a column is empty.
- `get_next_open_row(board, col)` returns the first empty row in a column.
- `drop_piece(board, row, col, piece)` places a piece on the board.


In [11]:
def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r

def drop_piece(board, row, col, piece):
    board[row][col] = piece


### Winning Move Detection
Checks horizontal, vertical, and diagonal sequences of 4 for a given piece.


In [12]:
def winning_move(board, piece):
    # Horizontal
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if all(board[r][c+i] == piece for i in range(4)):
                return True
    # Vertical
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c] == piece for i in range(4)):
                return True
    # Positive diagonal
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c+i] == piece for i in range(4)):
                return True
    # Negative diagonal
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if all(board[r-i][c+i] == piece for i in range(4)):
                return True
    return False


### Scoring Mechanism for AI
- `evaluate_window(window, piece)` evaluates a window of 4 cells and gives a score based on AI advantage.
- `score_position(board, piece)` sums up scores horizontally, vertically, and diagonally to evaluate the whole board.


In [13]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = AI1_PIECE if piece == AI2_PIECE else AI2_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

def score_position(board, piece):
    score = 0
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
    score += center_array.count(piece) * 3

    # Horizontal
    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMN_COUNT-3):
            score += evaluate_window(row_array[c:c+WINDOW_LENGTH], piece)
    # Vertical
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROW_COUNT-3):
            score += evaluate_window(col_array[r:r+WINDOW_LENGTH], piece)
    # Positive diagonal
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)
    # Negative diagonal
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)
    return score


### Valid Moves and Terminal State
- `get_valid_locations(board)` returns a list of columns where a piece can be dropped.
- `is_terminal_node(board)` checks if the game is over (win or draw).


In [14]:
def get_valid_locations(board):
    return [col for col in range(COLUMN_COUNT) if is_valid_location(board, col)]

def is_terminal_node(board):
    return winning_move(board, AI1_PIECE) or winning_move(board, AI2_PIECE) or len(get_valid_locations(board)) == 0


### Minimax AI with Alpha-Beta Pruning
- Recursive function to simulate future moves.
- `maximizingPlayer` chooses the best move for the AI.
- `minimizingPlayer` simulates opponent's response.


In [15]:
def minimax(board, depth, alpha, beta, maximizingPlayer, piece):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, piece):
                return (None, 1000000)
            elif winning_move(board, AI1_PIECE if piece == AI2_PIECE else AI2_PIECE):
                return (None, -1000000)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, piece))

    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, piece)
            new_score = minimax(b_copy, depth-1, alpha, beta, False, piece)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value
    else:
        value = math.inf
        column = random.choice(valid_locations)
        opp_piece = AI1_PIECE if piece == AI2_PIECE else AI2_PIECE
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, opp_piece)
            new_score = minimax(b_copy, depth-1, alpha, beta, True, piece)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value


### Main Console Loop for AI vs AI Game
- Randomly choose which AI starts.
- Each AI uses `minimax` to choose the best move.
- Game continues until a win or draw.


In [16]:
board = create_board()
print_board(board)
game_over = False
turn = random.randint(AI1, AI2)

while not game_over:
    if turn == AI1:
        col, _ = minimax(board, 5, -math.inf, math.inf, True, AI1_PIECE)
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, AI1_PIECE)
            print(f"AI 1 plays column {col}")
            print_board(board)
            if winning_move(board, AI1_PIECE):
                print("AI 1 wins!!")
                game_over = True
    else:
        col, _ = minimax(board, 5, -math.inf, math.inf, True, AI2_PIECE)
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, AI2_PIECE)
            print(f"AI 2 plays column {col}")
            print_board(board)
            if winning_move(board, AI2_PIECE):
                print("AI 2 wins!!")
                game_over = True

    if len(get_valid_locations(board)) == 0 and not game_over:
        print("Draw!")
        game_over = True

    turn += 1
    turn %= 2


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
AI 2 plays column 3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]]
AI 1 plays column 3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]]
AI 2 plays column 3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]]
AI 1 plays column 2
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
AI 2 plays column 3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
AI 1 plays column 2
[[0. 0. 0.

## <center> Human VS AI

# 🧩 1️⃣ Imports
# Idea:
# - numpy 👉 represents the board as a matrix
# - random 👉 choose a random move sometimes
# - math 👉 use +∞ and -∞ in minimax

In [17]:
import numpy as np
import random
import math

# 🎮 2️⃣ Game Constants
# Idea:
# - Board = 6 rows × 7 columns
# - Player = 0, AI = 1
# - EMPTY = 0, pieces = 1 or 2
# - WINDOW_LENGTH = 4 (win condition)

In [18]:
ROW_COUNT = 6
COLUMN_COUNT = 7
PLAYER = 0
AI = 1
EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2
WINDOW_LENGTH = 4

# 🧱 3️⃣ Create Board
# Idea: Initialize board with zeros, each 0 = empty

In [19]:
def create_board():
    return np.zeros((ROW_COUNT, COLUMN_COUNT))

# ⬇️ 4️⃣ Drop Piece
# Idea: Place the piece in the correct row & column

In [20]:
def drop_piece(board, row, col, piece):
    board[row][col] = piece

# ✅ 5️⃣ Valid Location
# Idea: Check if the top row in the column is empty → valid move

In [21]:
def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

# 📍 6️⃣ Next Open Row
# Idea: Find the first empty row in a column from bottom to top

In [22]:
def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r

# 🖨️ 7️⃣ Print Board
# Idea: Flip vertically so it looks like a real board

In [23]:
def print_board(board):
    print(np.flip(board, 0))

# 📊 8️⃣ Valid Columns
# Idea: Return a list of columns that are not full

In [24]:
def get_valid_locations(board):
    return [c for c in range(COLUMN_COUNT) if is_valid_location(board, c)]


# 🏆 9️⃣ Winning Move
# Idea: Check 4 pieces in all directions: horizontal, vertical, diagonals

In [25]:
def winning_move(board, piece):
    # Horizontal
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if all(board[r][c+i] == piece for i in range(4)):
                return True
    # Vertical
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c] == piece for i in range(4)):
                return True
    # Diagonal /
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c+i] == piece for i in range(4)):
                return True
    # Diagonal \
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if all(board[r-i][c+i] == piece for i in range(4)):
                return True
    return False

# 🤝 1️⃣0️⃣ Draw
# Idea: No valid moves left and no winner → Draw

In [26]:
def is_draw(board):
    return len(get_valid_locations(board)) == 0 and \
           not winning_move(board, PLAYER_PIECE) and \
           not winning_move(board, AI_PIECE)

# 🧠 1️⃣1️⃣ Terminal Node
# Idea: Game over if Player wins, AI wins, or Draw

In [27]:
def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or \
           winning_move(board, AI_PIECE) or \
           len(get_valid_locations(board)) == 0

# ⭐ 1️⃣2️⃣ Evaluate Window (4 cells)
# Idea: AI evaluates sequences of 4 cells
# - 4 AI pieces → very good
# - 3 AI pieces + 1 empty → good
# - 3 player pieces + 1 empty → subtract points

In [28]:

def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

# 📈 1️⃣3️⃣ Score Board
# Idea: Evaluate entire board
# - Center column is more valuable
# - Check all rows, columns, diagonals

In [29]:
def score_position(board, piece):
    score = 0
    # Center column
    center_array = list(board[:, COLUMN_COUNT//2])
    score += center_array.count(piece) * 3
    # Horizontal
    for r in range(ROW_COUNT):
        row_array = list(board[r,:])
        for c in range(COLUMN_COUNT-3):
            score += evaluate_window(row_array[c:c+4], piece)
    # Vertical
    for c in range(COLUMN_COUNT):
        col_array = list(board[:,c])
        for r in range(ROW_COUNT-3):
            score += evaluate_window(col_array[r:r+4], piece)
    # Diagonals
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            score += evaluate_window([board[r+i][c+i] for i in range(4)], piece)
            score += evaluate_window([board[r+3-i][c+i] for i in range(4)], piece)
    return score

# 🤖 1️⃣4️⃣ Minimax + Alpha-Beta
# Idea: AI looks ahead certain depth, tries all moves, picks best using pruning

In [30]:

def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)

    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return None, 10**14
            elif winning_move(board, PLAYER_PIECE):
                return None, -10**14
            else:
                return None, 0
        else:
            return None, score_position(board, AI_PIECE)

    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value
    else:
        value = math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER_PIECE)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

# ▶️ 1️⃣5️⃣ Main Game Loop
# Idea: Initialize board, alternate turns, check win/draw

In [ ]:
board = create_board()
game_over = False
turn = random.randint(PLAYER, AI)
print_board(board)

while not game_over:
    if turn == PLAYER:
        col = int(input("Player 1 Make your Selection (0-6): "))
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, PLAYER_PIECE)
            if winning_move(board, PLAYER_PIECE):
                print_board(board)
                print("PLAYER 1 WINS!!")
                game_over = True
            elif is_draw(board):
                print_board(board)
                print("DRAW!")
                game_over = True
            turn = AI
            print_board(board)
    else:
        col, _ = minimax(board, 5, -math.inf, math.inf, True)
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, AI_PIECE)
            if winning_move(board, AI_PIECE):
                print_board(board)
                print("AI WINS!!")
                game_over = True
            elif is_draw(board):
                print_board(board)
                print("DRAW!")
                game_over = True
            turn = PLAYER
            print_board(board)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


## <center> Human VS Human

### Import Libraries and Define Constants
- `numpy` is used to create and manipulate the game board.
- We define the board size, piece values, and player identifiers.


In [1]:
import numpy as np

# ================= CONSTANTS =================
ROW_COUNT = 6
COLUMN_COUNT = 7

EMPTY = 0
PLAYER1_PIECE = 1
PLAYER2_PIECE = 2


### Board Creation and Display
- `create_board()` initializes the board with zeros.
- `print_board()` prints the board flipped vertically for easier reading.


In [2]:
def create_board():
    return np.zeros((ROW_COUNT, COLUMN_COUNT), dtype=int)

def print_board(board):
    print(np.flip(board, 0))


### Dropping Pieces and Checking Valid Moves
- `drop_piece(board, row, col, piece)` places a piece in the board.
- `is_valid_location(board, col)` checks if the top row of a column is empty.
- `get_next_open_row(board, col)` returns the first empty row in the chosen column.


In [3]:
def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r


### Check for a Winning Move
- Checks all possible four-in-a-row sequences:
  - Horizontal
  - Vertical
  - Positive diagonal (\)
  - Negative diagonal (/)


In [4]:
def winning_move(board, piece):
    # Horizontal
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if all(board[r][c+i] == piece for i in range(4)):
                return True
    # Vertical
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c] == piece for i in range(4)):
                return True
    # Positive diagonal
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c+i] == piece for i in range(4)):
                return True
    # Negative diagonal
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if all(board[r-i][c+i] == piece for i in range(4)):
                return True
    return False


### Check for a Draw
- `is_draw(board)` returns True if all columns are full and no player has won.


In [6]:
def is_draw(board):
    return all(not is_valid_location(board, c) for c in range(COLUMN_COUNT))


### Main Game Loop
- Alternates turns between Player 1 and Player 2.
- Accepts column input from players.
- Updates the board and prints it.
- Checks for a win or draw after each move.


In [9]:
board = create_board()
game_over = False
turn = 0  # 0 -> Player 1, 1 -> Player 2

print_board(board)

while not game_over:
    player = "Player 1" if turn == 0 else "Player 2"
    piece = PLAYER1_PIECE if turn == 0 else PLAYER2_PIECE

    # Ask for player input
    valid_move = False
    while not valid_move:
        try:
            col = int(input(f"{player} - Choose a column (0-{COLUMN_COUNT-1}): "))
            if 0 <= col < COLUMN_COUNT and is_valid_location(board, col):
                valid_move = True
            else:
                print("Invalid column. Try again.")
        except ValueError:
            print("Invalid input. Enter an integer.")

    row = get_next_open_row(board, col)
    drop_piece(board, row, col, piece)

    print_board(board)

    if winning_move(board, piece):
        print(f"{player} WINS!")
        game_over = True
    elif is_draw(board):
        print("DRAW!")
        game_over = True
    else:
        turn = (turn + 1) % 2



[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


Player 1 - Choose a column (0-6):  1


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0]]


Player 2 - Choose a column (0-6):  6


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 0 0 0 0 2]]


...

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 1 1 0 2 2]]


Player 2 - Choose a column (0-6):  0


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [2 1 1 1 0 2 2]]


Player 1 - Choose a column (0-6):  4


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [2 1 1 1 1 2 2]]
Player 1 WINS!
